In [1]:
# Import numpy and opencv
import numpy as np
import cv2

# Define a function to perform red channel compensation
def red_channel_compensation(image, alpha, window_size):
    # Convert the image to float32
    image = image.astype(np.float32)
    # Split the image into RGB channels
    B, G, R = cv2.split(image)
    # Initialize the compensated red channel
    R_comp = np.zeros_like(R)
    # Get the image height and width
    height, width = image.shape[:2]
    # Get the half window size
    half = window_size // 2
    # Pad the image borders with zeros
    image = cv2.copyMakeBorder(image, half, half, half, half, cv2.BORDER_CONSTANT, value=0)
    # Loop over the image pixels
    for i in range(height):
        for j in range(width):
            # Get the local window of the image
            window = image[i:i+window_size, j:j+window_size]
            # Get the mean value of the green and blue channels in the window
            mean_G = np.mean(window[:, :, 1])
            mean_B = np.mean(window[:, :, 2])
            # Compute the compensated red value according to equation (8) in the paper
            R_comp[i, j] = R[i, j] + (alpha * mean_G + (1 - alpha) * mean_B - R[i, j]) * (alpha * G[i, j] + (1 - alpha) * B[i, j]) / (R[i, j] + G[i, j] + B[i, j])
    # Clip the compensated red channel to the range [0, 255]
    R_comp = np.clip(R_comp, 0, 255)
    # Merge the RGB channels
    image = cv2.merge((B, G, R_comp))
    # Convert the image back to uint8
    image = image.astype(np.uint8)
    # Return the compensated image
    return image


In [2]:
# Assuming the input image is in RGB format and stored in a variable named img
import numpy as np
import cv2

# Define a function for white balance based on the maximal response assumption
def white_balance(img):
    # Get the maximum values of each channel
    r_max = np.max(img[:,:,0])
    g_max = np.max(img[:,:,1])
    b_max = np.max(img[:,:,2])

    # Compute the scaling factors for each channel
    r_scale = r_max / 255
    g_scale = g_max / 255
    b_scale = b_max / 255

    # Apply the scaling factors to the image
    img[:,:,0] = img[:,:,0] / r_scale
    img[:,:,1] = img[:,:,1] / g_scale
    img[:,:,2] = img[:,:,2] / b_scale

    # Clip the values to the range [0, 255]
    img = np.clip(img, 0, 255)

    # Convert the image to uint8 type
    img = img.astype(np.uint8)

    # Return the white balanced image
    return img


# Example of using the function
# output_image = white_balance(r"D:\mini project\practise\compensated_image.jpg")
# output_image_path = "D:\mini project\practise\white_balanced_image.jpg"
# cv2.imwrite(output_image_path, output_image)
# print(f"Compensated image saved as {output_image_path}")

In [3]:
# Import numpy and opencv
import numpy as np
import cv2

# Define a function to implement illuminance improvement by retinex with dense pixels
def retinex_with_dense_pixels(image):
    # Convert the image from BGR to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Extract the value component
    v = hsv[:, :, 2]
    # Get the height and width of the image
    h, w = v.shape
    # Initialize the illumination component
    l = np.zeros_like(v, dtype=np.float32)
    # Define the eight directions for the path selection
    directions = [(1, 0), (0, 1), (-1, 0), (0, -1), (1, 1), (-1, -1), (1, -1), (-1, 1)]
    # Define the step size for each iteration
    step = 1
    # Define the number of iterations
    n_iter = int(np.log2(min(h, w)))
    # Loop over the eight directions
    for d in directions:
        # Loop over the iterations
        for i in range(n_iter):
            # Calculate the offset for the current iteration
            offset = step * (2 ** i)
            # Loop over the rows of the image
            for x in range(h):
                # Loop over the columns of the image
                for y in range(w):
                    # Calculate the coordinates of the next pixel along the direction
                    x_next = x + d[0] * offset
                    y_next = y + d[1] * offset
                    # Check if the coordinates are valid
                    if 0 <= x_next < h and 0 <= y_next < w:
                        # Update the illumination component by comparing the pixel values
                        l[x, y] = max(l[x, y], v[x, y] - v[x_next, y_next])
    # Average the illumination component over the eight directions
    l = l / 8
    # Normalize the illumination component to [0, 255]
    l = cv2.normalize(l, None, 0, 255, cv2.NORM_MINMAX)
    # Calculate the reflection component by subtracting the illumination component from the value component
    r = v - l
    # Replace the value component with the reflection component
    hsv[:, :, 2] = r
    # Convert the image back to BGR color space
    result = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    # Return the result
    return result


In [4]:
# Define a function for adaptive histogram transform[^1^][1]
def adaptive_histogram_transform(image):
    # Convert the image from RGB to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    # Split the HSV channels
    h, s, v = cv2.split(hsv)
    # Get the mean value of the V channel
    mean_v = np.mean(v)
    # Define the epsilon value
    epsilon = 10
    # Define the piecewise linear function for histogram transform[^2^][2]
    def transform(x):
        if x < mean_v - epsilon:
            return 100 * x / (mean_v - epsilon)
        elif abs(x - mean_v) <= epsilon:
            return x - mean_v + 120
        else:
            return 115 + 140 * (x - mean_v - epsilon) / (255 - mean_v - epsilon)
    # Apply the transform function to the V channel
    v = np.vectorize(transform)(v)
    # Clip the values to the range [0, 255]
    v = np.clip(v, 0, 255).astype(np.uint8)
    # Merge the HSV channels
    hsv = cv2.merge([h, s, v])
    # Convert the image back to RGB color space
    image = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    # Return the transformed image
    return image


In [7]:
image_path = r"D:\miniproject\image-processing-dataset\raw-890\41_img_.png"  # Replace with your own image path
image = cv2.imread(image_path)
alpha = 1  # You can adjust this value
window_size = 5  # You can adjust the window size
compensated_image = red_channel_compensation(image, alpha, window_size)
white_balanced_image = white_balance(compensated_image)
retinex_image = retinex_with_dense_pixels(white_balanced_image)
final_image = adaptive_histogram_transform(retinex_image)

output_path = "D:\miniproject\practise\processed_image.jpg"
cv2.imwrite(output_path, compensated_image)

print(f"Final output image saved to {image_path}. Processed image saved as {output_path}.")

C:\Users\Dell\AppData\Local\Temp\ipykernel_21056\4070715851.py:37: RuntimeWarning: overflow encountered in ubyte_scalars
  l[x, y] = max(l[x, y], v[x, y] - v[x_next, y_next])


Final output image saved to D:\miniproject\image-processing-dataset\raw-890\41_img_.png. Processed image saved as D:\miniproject\practise\processed_image.jpg.
